## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-22-22-53-09 +0000,nypost,"3 inmates, including murder suspect, escape fr...",https://nypost.com/2025/12/22/us-news/atlanta-...
1,2025-12-22-22-52-13 +0000,nyt,"Betty Reid Soskin, Nation’s Oldest Park Ranger...",https://www.nytimes.com/2025/12/21/us/betty-re...
2,2025-12-22-22-50-20 +0000,nypost,Odds of winning Monday’s monster $1.6B Powerba...,https://nypost.com/2025/12/22/us-news/here-are...
3,2025-12-22-22-46-56 +0000,nypost,"ICE rounds up over 1,000 illegal immigrants du...",https://nypost.com/2025/12/22/us-news/ice-roun...
4,2025-12-22-22-44-35 +0000,nypost,Long Island’s largest soup kitchen now helping...,https://nypost.com/2025/12/22/us-news/long-isl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2381/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
85,trump,30
236,new,16
107,christmas,12
185,general,10
32,up,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
126,2025-12-22-16-52-46 +0000,nypost,Trump admin scraps 5 offshore wind farms — inc...,https://nypost.com/2025/12/22/us-news/trump-ad...,77
139,2025-12-22-15-49-12 +0000,bbc,New Trump envoy says he will serve to make Gre...,https://www.bbc.com/news/articles/ckgmd132ge4o...,74
83,2025-12-22-19-16-47 +0000,nypost,Bill Clinton accuses Trump’s DOJ of cover-up i...,https://nypost.com/2025/12/22/us-news/bill-cli...,72
146,2025-12-22-15-34-01 +0000,nypost,Trump approval rating rebounds as new poll sho...,https://nypost.com/2025/12/22/us-news/trump-ap...,65
47,2025-12-22-21-14-32 +0000,nypost,President Trump to announce new class of battl...,https://nypost.com/2025/12/22/us-news/presiden...,64


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
126,77,2025-12-22-16-52-46 +0000,nypost,Trump admin scraps 5 offshore wind farms — inc...,https://nypost.com/2025/12/22/us-news/trump-ad...
142,56,2025-12-22-15-39-36 +0000,wapo,Car bomb in Moscow kills senior Russian genera...,https://www.washingtonpost.com/world/2025/12/2...
56,45,2025-12-22-20-39-00 +0000,wsj,Former Vice President Mike Pence’s political g...,https://www.wsj.com/politics/elections/top-her...
158,44,2025-12-22-14-47-26 +0000,latimes,Larry Ellison pledges $40-billion personal gua...,https://www.latimes.com/entertainment-arts/bus...
83,39,2025-12-22-19-16-47 +0000,nypost,Bill Clinton accuses Trump’s DOJ of cover-up i...,https://nypost.com/2025/12/22/us-news/bill-cli...
157,37,2025-12-22-14-49-17 +0000,nypost,Jimmy Kimmel to make Christmas Day address on ...,https://nypost.com/2025/12/22/media/jimmy-kimm...
236,37,2025-12-22-06-38-22 +0000,nypost,Bondi Beach terrorists railed against ‘Zionist...,https://nypost.com/2025/12/22/world-news/polic...
86,34,2025-12-22-19-01-17 +0000,nyt,"What to Know About Louisiana Gov. Jeff Landry,...",https://www.nytimes.com/2025/12/22/us/jeff-lan...
229,32,2025-12-22-08-31-18 +0000,wsj,"Stock Market Today: S&P 500 Rises; Gold, Silve...",https://www.wsj.com/livecoverage/stock-market-...
140,30,2025-12-22-15-49-00 +0000,wsj,CBS News Editor Bari Weiss Defends Decision to...,https://www.wsj.com/business/media/cbs-news-ed...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
